# vector swap



In [1]:
from generate import *
from closest_vs_minimal import *

iterátor for, prochazim postupne floored vector 0
iterátor for, prochazim postupne floored vector 1
[7.0, 10.0, 15.0] inside if
[9.0, 11.0, 16.0] inside if
[10.0, 14.0, 17.0] inside if
[11.0, 15.0, 21.0]
nav [(11.0, 15.0, 21.0)]
iterátor for, prochazim postupne floored vector 2
[7.0, 10.0, 15.0] inside if
[9.0, 11.0, 16.0] inside if
[10.0, 14.0, 17.0] inside if
[11.0, 15.0, 21.0]
nav [(11.0, 15.0, 21.0), (11.0, 15.0, 21.0)]
efficient returned:  [(11.0, 15.0, 21.0), (11.0, 15.0, 21.0)]
(7, 10, 15)
(8, 13, 16)
(8, 11, 19)
(9, 14, 20)


In [2]:
# def swapping(basis_as_matrix):
#     basis_as_list= matrix_to_list(basis_as_matrix)
#     dim = len(basis_as_list)
#     # print("B = ",basis_as_list)
#     used_vectors =[[0]*dim]
#     used_vectors.extend(basis_as_list)

#     G = gram_matrix(basis_as_matrix)

#     basis_as_list.sort(key=lambda x: euclidean_norm(x))
#     # print("Basis na začatku: ")
#     # for i in range(dim):
#     #         print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
#     # print("--------------")
#     basis_as_matrix=matrix(basis_as_list)
#     # print(basis_as_matrix)
#     # print(basis_as_list)

#     i = 0
#     while i < dim:
# #     for j in range(0,dimension):
#         # print("\n")
#         # print(i)
#         real_min = vector_to_list(find_real_minimum_genfile(G, i%dim, 1))

#         # print(evaluate_norms_at_cube(cube_points(real_min), basis_as_list))
        
#         shortestLC = shortest_lc_in_cube(real_min, basis_as_matrix)
#         # print(real_min)
#         # print(shortestLC)
#         shortest_vector = vector(shortestLC)*basis_as_matrix
#         # print("sv", shortest_vector,(shortest_vector).norm().n())
#         # print("used vectors",used_vectors)
#         if vector_to_list(shortest_vector) in used_vectors:
#               break
#             #   print("ERROR: I already used the vector ", shortest_vector)
#               i+=1
#               continue
#         if shortest_vector.norm() > basis_as_matrix[-1].norm():
#               break
#             #   print("ERROR, LONGER VECTOR")
#               i+=1
#               continue
#         # print(type(shortest_vector))
#         shortest_vector = vector_to_list(shortest_vector)
#         used_vectors.append(shortest_vector)
#         # print("basis to be thrown away", basis_as_list[dimension-1])
#         basis_as_list[dim-1] = shortest_vector
#         # print(basis_as_list)
#         # print("basis new, should be SV", basis_as_list[dimension-1])
#         basis_as_list.sort(key=lambda x: euclidean_norm(x))
#         # print(basis_as_list)
#         # for i in range(dim):
#                 # print("  b_",i, " ", euclidean_norm(basis_as_list[i]), basis_as_list[i])
#         basis_as_matrix = matrix(basis_as_list)
#         G = gram_matrix(basis_as_matrix)
#     # print("reduced lll",  (matrix_to_list(matrix(ZZ, basis_as_list).LLL())), euclidean_norm(first_nonzero_norm_vector(matrix_to_list(matrix(ZZ, basis_as_list).LLL()))))
#     return euclidean_norm(basis_as_list[0])


    
# # možná se ještě vyplatí opravit cube takto: co když sii jako real-min uložím něco, co ale je moc daleko a  
# # fixací JINÉ souřadnice ale dostanu už správnej výsledek? ještě na to mrkni viko
# # tot je na opravu funkce compare with cube

In [3]:
def gaussian_reduction(b_1, b_2):
    """Input: vectos as a list
    Output: sage vectors [longer, shorter]
    """
    b_1 = vector(b_1)
    b_2 = vector(b_2)
    if b_1.norm() < b_2.norm():
        b_1 , b_2 = b_2, b_1
    r = round(b_1.dot_product(b_2) / b_2.norm()**2)
    a = b_1 - r * b_2
    if a.norm() < b_2.norm():
        a, b_2 = gaussian_reduction(a, b_2)
    return a, b_2


In [4]:
from sage.modules.free_module_integer import IntegerLattice
from generate import *

def vector_to_int_list(vector) -> list:
    return [int(num) for num in vector.list()]

def semaev(basis):
    dim = basis.nrows()

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    iterace = 0
    shortest_vectors_sequence = []

    while True:
        shortest_vectors_sequence.append(basis[0].norm().n())
        iterace += 1
        # print(basis)
        # reduce the pair b_1 and b_2
        basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])


        # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis)
        lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
        slc = shortest_lc_in_cube(lc_cube, basis)
        candidate = vector(ZZ,slc) * basis
        # candidate_norm = sqrt((vector(slc)*G*matrix(slc)-transpose())[0]).n()
        # print(candidate_norm, candidate.norm().n(), "slcG slc.T vs kandidat norma, melo by byt stejne")

        # if I found a shorter vector candidate, continue
        longest_norm = sqrt(G[dim-1,dim-1]).n()
        # print(candidate.norm().n() < longest_norm , (candidate) not in basis, candidate.norm(), longest_norm)
        if candidate.norm() < longest_norm and (candidate) not in basis:
            basis[dim-1] = candidate
            sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
            basis = basis[sorted_indices, :]
            continue
        break
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    return basis, iterace

def find_real_minimum_mistni(G, current_row, inserted_component) -> vector:
    dimension = len(matrix_to_list(G))
    matrixA = matrix(dimension - 1, dimension - 1, 0) # square matrix of size (dimension - 1) x (dimension - 1), filled with zeros.
    matrixB = matrix(dimension - 1, 1, 0) # column matrix of size (dimension - 1) x 1, filled with zeros.
    matrixA[0,0] = 1
    a, b = 0, 0
    for row in range(dimension):
        if row != current_row:
            matrixA[a] = [G[row, j] for j in range(len(G[row])) if j != current_row]
            matrixB[b] = sum([inserted_component * G[row,j] for j in range(len(G[row])) if j == current_row])
            a += 1
            b += 1
    # insert indices
    result = (matrixA.solve_right(-(inserted_component) * matrixB)).list()
    result.insert(current_row, inserted_component)
    return vector(result).n()

# print(counter(3 , 1000000000000000000000, 100, semaev))




In [18]:
# # TADY POČÍTÁM POČET ITERACÍ VERSUS VELIKOST ČÍSEL MATII 

for exp in range(6,40,4):
    avg = 0
    iterations = 5
    prime = random_prime(10^6, True, 10)
    for j in range(iterations):
        avg += semaev(hard_instance_lattice(n=6, r=1, q=prime))[1]
        # avg += semaev(random_special_matrix(3, 10^exp))[1]
    print("Range čísel v prvním sloupci: deset na ", exp, "-> average počet iterací", float(avg/iterations))


Range čísel v prvním sloupci: deset na  6 -> average počet iterací 6.0
Range čísel v prvním sloupci: deset na  10 -> average počet iterací 5.4
Range čísel v prvním sloupci: deset na  14 -> average počet iterací 5.4
Range čísel v prvním sloupci: deset na  18 -> average počet iterací 4.8
Range čísel v prvním sloupci: deset na  22 -> average počet iterací 4.4
Range čísel v prvním sloupci: deset na  26 -> average počet iterací 4.2
Range čísel v prvním sloupci: deset na  30 -> average počet iterací 4.8
Range čísel v prvním sloupci: deset na  34 -> average počet iterací 4.2
Range čísel v prvním sloupci: deset na  38 -> average počet iterací 4.4


In [6]:
from hardinstancelattice import hard_instance_lattice

#semaev(hard_instance_lattice(n=4, r=1, q=113))

[  1   0   0 112]
[  0   1   0  85]
[  0   0   1  27]


In [42]:
# ZDE POČÍTÁM ÚSPĚŠNOST  +  RUNTIME SEMAIEVA


from math import isclose
import time

def counter(dimension : int, perimeter, tries, function):
    res = {"True":0, "False":0}
    total_time =0
    total_iterations = 0
    for _ in range(tries):
        mojematice = random_special_matrix(dimension, perimeter)
        # mojematice = randomMatrix(dimension, perimeter)
        start = time.time()
        swapresult, this_it = function(mojematice)
        total_iterations += float(this_it)
        end = time.time()
        total_time += end - start
        this_function_closest_norm = euclidean_norm(matrix_to_list(swapresult)[0])
        L = IntegerLattice(mojematice, lll_reduce=False)
        sv_exact = L.shortest_vector()
        sv_exact_norm = sv_exact.norm().n()

        if  isclose(this_function_closest_norm, sv_exact_norm, abs_tol=1e-4):
            res["True"]+=1
            continue
        lllnorm = euclidean_norm(first_nonzero_norm_vector(matrix_to_list(mojematice.LLL())))
        # print(mojematice)
        print("falsuje", this_function_closest_norm,"exact: ", sv_exact_norm, "LLLko je ", lllnorm, "B = ", matrix_to_list(mojematice))
        res["False"]+=1
    print(total_time/tries, "is the total time, ", total_iterations/tries,"the average num of iterations  of method ", function)
    return res

In [8]:
def semaev_modified_for_higher_bases(basis_as_matrix):
    # print(basis_as_matrix)
    basis_as_list = matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    iterace = 0
    L = IntegerLattice(basis_as_matrix, lll_reduce=False)
    sv_exact = L.shortest_vector()
    sv_exact_norm = sv_exact.norm().n()
    # print(sv_exact_norm, "exact norm")
    while True:
        iterace += 1

        # Reduce the pair b_1 and b_2
        basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
        basis_as_list[1] = vector_to_int_list(basis_as_list[1])
        basis_as_list[0] = vector_to_int_list(basis_as_list[0])

        basis_as_matrix=matrix(ZZ,basis_as_list)

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis_as_matrix)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim - 1, 1))
        slc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
        a = vector(ZZ, slc) * basis_as_matrix

        # if I found a shorter vector a, continue
        if a.norm().n() < basis_as_matrix[dim-1].norm().n() and vector_to_int_list(a) != [0]*dim and vector_to_int_list(a) not in basis_as_list:
            basis_as_list[dim-1] = vector_to_int_list(a)
            basis_as_list.sort(key=lambda x: euclidean_norm(x))
            basis_as_matrix=matrix(ZZ,basis_as_list)
            # print("continue", a.norm().n() , basis_as_matrix[dim-1].norm().n())
            continue
        elif a.norm().n() >= basis_as_matrix[dim-1].norm().n(): # pokud to už nezkratim 
            # print("rekurzuju, aktualni sv ma normu ", basis_as_matrix[0].norm().n())
            basis_as_matrix[0:dim-1] = matrix(semaev(matrix(basis_as_list[0:dim-1]))[0]) # FIXME dat zpět trojku
        # else:
            # print(a.norm().n() < basis_as_matrix[2].norm().n() , vector_to_int_list(a) != [0]*dim , vector_to_int_list(a) not in basis_as_list)
        break
    # print(iterace, "iterace")
    return basis_as_matrix, iterace




In [9]:
def first_nonzero_norm_vector(vectors):
    for v in vectors:
        if euclidean_norm(v) != 0:
            return v

In [10]:
def semaev_with_rounding(basis_as_matrix):
    # print(basis_as_matrix)
    basis_as_list= matrix_to_list(basis_as_matrix)
    dim = len(basis_as_list)
    basis_as_list.sort(key=lambda x: euclidean_norm(x))
    iterace = 0
    shortest_vectors_sequence = []
    while True:
        shortest_vectors_sequence.append(basis_as_matrix[0].norm().n())
        iterace += 1
        # Reduce the pair b_1 and b_2
        basis_as_list[1], basis_as_list[0] = gaussian_reduction(basis_as_list[1], basis_as_list[0])
        basis_as_list[1] = vector_to_int_list(basis_as_list[1])
        basis_as_list[0] = vector_to_int_list(basis_as_list[0])

        basis_as_matrix=matrix(ZZ,basis_as_list)

        # find a minimum of |a|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
        G = gram_matrix(basis_as_matrix)
        lc_cube = vector_to_list(find_real_minimum_mistni(G, dim-1, 1))
        slc = vector([round(x) for x in lc_cube])
        # slc = shortest_lc_in_cube(lc_cube, basis_as_matrix)
        # print("slc", slc)
        a = vector(ZZ,slc) * basis_as_matrix
        # if I found a shorter vector a, continue
        if a.norm().n() < basis_as_matrix[dim-1].norm().n() and vector_to_int_list(a) != [0]*dim and vector_to_int_list(a) not in basis_as_list:
            basis_as_list[dim-1] = (a)
            basis_as_list.sort(key=lambda x: euclidean_norm(x))
            # for basevector in basis_as_list:
            #     print(euclidean_norm(basevector))
            # print("-")
            basis_as_matrix=matrix(ZZ,basis_as_list)
            # print(basis_as_matrix[0].norm().n())
            continue
        break
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    return basis_as_matrix, iterace


In [49]:
def jeden_semaev_a_pak_LLL(basis):
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    # -- - - - - - SEMAEV

    # reduce the pair b_1 and b_2
    basis[1], basis[0] = gaussian_reduction(basis[1], basis[0])

    # find a minimum of |candidate|=|b_3 +x_2b_2 + x_1b_1| with regard to x_1, x_2
    G = gram_matrix(basis)
    lc_cube = find_real_minimum_mistni(G, dim-1, 1) # FIXME might break for nonsquare matrices
    slc = shortest_lc_in_cube(lc_cube, basis)
    candidate = vector(ZZ,slc) * basis

    # rovnou nahrazuju

    basis[dim-1] = candidate

    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]
    
    # print("předlll")
    # for vec in basis:
    #     print(vec.norm().n())
        
    basis = basis.LLL()

    # print("po LLL")
    # for vec in basis:
    #     print(vec.norm().n())
        
    # basis = basis.LLL()
    # print(iterace, "iterace")
    # print("poměry")
    # print(*[f"{shortest_vectors_sequence[i]/shortest_vectors_sequence[i+1]}" for i in range(len(shortest_vectors_sequence)-1)], sep="\n")
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :]    

    return basis, 0

jeden_semaev_a_pak_LLL(random_special_matrix(3,10000))


(
[ -1  -3  10]   
[-10  -6  -6]   
[  8 -14   1], 0
)

In [40]:
def vestavene_LLL(basis):
    dim = basis.nrows()
    basis = basis.LLL()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return basis, 0
    

In [31]:
from olll import full_LLL
from generate import random_special_matrix

def poradne_LLL(basis):
    basis, it = full_LLL(basis)
    dim = basis.nrows()
    sorted_indices = sorted(range(dim), key=lambda i: basis[i].norm())
    basis = basis[sorted_indices, :] 
    return matrix(basis), it

# poradne_LLL(random_special_matrix(3,1000))

In [55]:
# print(counter(3, 100000000000, 10, jeden_semaev_a_pak_LLL))
# # print(counter(6, 100000000000, 100, semaev))
# # print(counter(6, 100000000000, 100, semaev_with_rounding))
# print(counter(3, 100000000000, 10, vestavene_LLL))
# # print()
# print(counter(4, 100000000000, 100, jeden_semaev_a_pak_LLL))
# # print(counter(4, 100000000000, 100, semaev_modified_for_higher_bases))
# print(counter(4, 100000000000, 100, vestavene_LLL))
# # print(counter(4, 100000000000, 100, poradne_LLL))
# # print()

# print(counter(6, 100000000000, 100, jeden_semaev_a_pak_LLL))
# # # print(counter(6, 1000000, 100, semaev))
# # # print(counter(6, 1000000, 100, semaev_modified_for_higher_bases))
# print(counter(6, 100000000000, 100, vestavene_LLL))

# # print()
# print(counter(10, 100000000000, 10, jeden_semaev_a_pak_LLL))
# # # print(counter(10, 1000000, 10, semaev))
# # # print(counter(10, 1000000, 10, semaev_modified_for_higher_bases))
# print(counter(10, 100000000000, 10, vestavene_LLL))

print(counter(40, 100000000000, 10, jeden_semaev_a_pak_LLL))
# # print(counter(10, 1000000, 10, semaev))
# # print(counter(10, 1000000, 10, semaev_modified_for_higher_bases))
print(counter(40, 100000000000, 10, vestavene_LLL))

falsuje 3.3166247903554 exact:  3.00000000000000 LLLko je  3.0 B =  [[-33381428034, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [13458797548, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [45213208634, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [33158263993, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [6843541206, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [48561993010, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [-80603330543, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [91532